# Natural Language Project - Finding the Genre of Movie Plots - Group 11

## Project Setup
If you're running this project in **Google Colab**, make sure to execute the following commands to properly configure the environment.
(These steps are not required if you're running the project locally on your machine.)

In [1]:
!git clone https://github.com/rspecker/NLP.git
%cd NLP

Cloning into 'NLP'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 289 (delta 16), reused 12 (delta 5), pack-reused 258 (from 1)
Receiving objects: 100% (289/289), 16.95 MiB | 16.86 MiB/s, done.
Resolving deltas: 100% (167/167), done.
/content/NLP


In [2]:
# !git pull

In [3]:
# !pip install -r requirements.txt

In [ ]:
! pip install sentence-transformers

In [18]:
# import project dependencies
import pandas as pd
from modelling.grid import classifiers
from modelling.grid_search import perform_grid_search
from modelling.evaluation import evaluate_model, save_results_to_file, save_confusion_matrix, save_test_data_with_predictions, apply_model_to_unlabeled_data
from preprocessing.embeddings import create_sentence_embeddings
from preprocessing.preproc import create_preprocesssed_dataset
from preprocessing.tfidf import create_train_data_tfidf
from utils import create_train_test_sets
from modelling.information_ret import score

## Data Import

In [6]:
# Import data and set column names
df = pd.read_table(
    'train.txt',
    names=['title', 'from', 'genre', 'director', 'plot']
    )

In [7]:
# Import data without labels and set column names
df_no_labels = pd.read_table(
    'test_no_labels.txt',
    names=['title', 'from', 'director', 'plot']
    )

## Data Pre-Processing

In [8]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = create_train_test_sets(
    df, test_size=0.2, random_state=0, y_column='genre'
)

### TF-IDF
TF-IDF converts text into sparse vectors based on word importance across documents, without considering word order or context

In [9]:
# create TF-IDF
vectorizer, x_train_tfidf = create_train_data_tfidf(x_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
vectorizer, x_test_tfidf = create_train_data_tfidf(x_test, vectorizer)

In [11]:
vectorizer, no_labels_tfidf = create_train_data_tfidf(df_no_labels, vectorizer)

### Feature-based Transfer Learning with Sentence Embeddings (Sentence-BERT)
Sentence-BERT generates dense, fixed-length embeddings that capture the semantic meaning of entire sentences, enabling more context-aware comparisons.

In [ ]:
# create sentence embeddings for testing data
x_train_sentence_embeddings = create_sentence_embeddings(
    sentences=x_train["plot"].to_list(),
    model="all-MiniLM-L6-v2")

In [13]:
# create sentence embeddings for testing data
x_test_sentence_embeddings = create_sentence_embeddings(
    sentences=x_test["plot"].to_list(),
    model="all-MiniLM-L6-v2")

In [14]:
# create sentence embeddings for unlabelled data
no_labels_sentence_embeddings = create_sentence_embeddings(
    sentences=df_no_labels["plot"].to_list(),
    model="all-MiniLM-L6-v2")

## Training

### TF-IDF

In [15]:
del classifiers["MultinomialNB"]

In [ ]:
for model_name, (model, param_grid) in classifiers.items():
    model_type="word_embeddings"
    # Perform GridSearchCV
    best_model, best_params, best_score = perform_grid_search(
        model, param_grid, model_type,model_name,
        x_train_tfidf, y_train)

    # Evaluate the model
    y_pred, test_accuracy, classification_rep, cm = evaluate_model(
        best_model, x_test_tfidf, y_test)

    # Save the results
    save_results_to_file(
        model_type, model_name, best_params, best_score, test_accuracy, classification_rep)

    save_test_data_with_predictions(x_test, y_test, y_pred, model_type, model_name)

    # Save confusion matrix as image
    save_confusion_matrix(model_type, model_name, cm, y_test)

    # Apply model to unlabeled data
    apply_model_to_unlabeled_data(best_model, no_labels_tfidf, model_type, model_name)

### Feature-based Transfer Learning with Sentence Embeddings (Sentence-BERT)

In [ ]:
# del classifiers["MultinomialNB"]

In [ ]:
for model_name, (model, param_grid) in classifiers.items():
    model_type="sentence_embeddings"
    # Perform GridSearchCV
    best_model, best_params, best_score = perform_grid_search(
        model, param_grid, model_type,model_name,
        x_train_sentence_embeddings, y_train)

    # Evaluate the model
    y_pred, test_accuracy, classification_rep, cm = evaluate_model(
        best_model, x_test_sentence_embeddings, y_test)

    # Save the results
    save_results_to_file(
        model_type, model_name, best_params, best_score, test_accuracy, classification_rep)
    save_test_data_with_predictions(x_test, y_test, y_pred, model_type, model_name)

    # Save confusion matrix as image
    save_confusion_matrix(model_type, model_name, cm, y_test)


    # Apply model to unlabeled data
    apply_model_to_unlabeled_data(best_model, no_labels_sentence_embeddings, model_type, model_name)

In [ ]:
warnings.filterwarnings("ignore")
# Import data
df = pd.read_table('train.txt',
                    names=['title', 'from', 'genre', 'director', 'plot'])
# Split data into training and testing sets
x_train, x_test, y_train, y_test = create_train_test_sets(
    df, test_size=0.2, random_state=0, y_column='genre'
)
score(x_train, x_test, y_train, y_test)